In [3]:
import pandas as pd

In [4]:
df=pd.DataFrame(pd.read_table('./dataset/SMSSpamCollection'))
df

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...
...,...,...
5566,spam,This is the 2nd time we have tried 2 contact u...
5567,ham,Will ü b going to esplanade fr home?
5568,ham,"Pity, * was in mood for that. So...any other s..."
5569,ham,The guy did some bitching but I acted like i'd...


In [5]:
column_names=['Spam/Not_Spam','Message']
df.columns=column_names
df

,Spam/Not_Spam,Message
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...
...,...,...
5566,spam,This is the 2nd time we have tried 2 contact u...
5567,ham,Will ü b going to esplanade fr home?
5568,ham,"Pity, * was in mood for that. So...any other s..."
5569,ham,The guy did some bitching but I acted like i'd...


In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Spam/Not_Spam  5571 non-null   str  
 1   Message        5571 non-null   str  
dtypes: str(2)
memory usage: 542.8 KB


In [7]:
df['Message'].duplicated().value_counts()

Message
False    5168
True      403
Name: count, dtype: int64

In [8]:
df = df.drop_duplicates(subset='Message')
df['Message'].duplicated().value_counts()

Message
False    5168
Name: count, dtype: int64

In [9]:
df.info()

<class 'pandas.DataFrame'>
Index: 5168 entries, 0 to 5570
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Spam/Not_Spam  5168 non-null   str  
 1   Message        5168 non-null   str  
dtypes: str(2)
memory usage: 539.2 KB


In [10]:
df['Spam/Not_Spam'].unique()

<ArrowStringArray>
['ham', 'spam']
Length: 2, dtype: str

In [11]:
mapping={'ham':1,'spam':0}
df['Result']=df['Spam/Not_Spam'].map(mapping)
df

,Spam/Not_Spam,Message,Result
0,ham,Ok lar... Joking wif u oni...,1
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0
2,ham,U dun say so early hor... U c already then say...,1
3,ham,"Nah I don't think he goes to usf, he lives aro...",1
4,spam,FreeMsg Hey there darling it's been 3 week's n...,0
...,...,...,...
5566,spam,This is the 2nd time we have tried 2 contact u...,0
5567,ham,Will ü b going to esplanade fr home?,1
5568,ham,"Pity, * was in mood for that. So...any other s...",1
5569,ham,The guy did some bitching but I acted like i'd...,1


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Prepare Features (X) and Labels (y)
X = df['Message']
y = df['Result'] 

# 2. Split into Training (80%) and Testing (20%) sets - USE STRATIFIED SPLIT
# stratify=y ensures train/test have same class distribution (important for imbalanced data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Vectorization: Convert text to a numerical matrix
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 4. Initialize and Train the SVM Model
model = SVC(kernel='linear') 
model.fit(X_train_tfidf, y_train)

# 5. Model Evaluation
predictions = model.predict(X_test_tfidf)

print("--- Model Performance (with Stratified Split) ---")
print(f"Accuracy Score: {accuracy_score(y_test, predictions):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, predictions))

--- Model Performance (with Stratified Split) ---
Accuracy Score: 0.99

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       131
           1       0.99      1.00      0.99       903

    accuracy                           0.99      1034
   macro avg       0.98      0.95      0.97      1034
weighted avg       0.99      0.99      0.99      1034



In [13]:
train_acc = model.score(X_train_tfidf, y_train)
test_acc = model.score(X_test_tfidf, y_test)

print(f"Training Accuracy: {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")


Training Accuracy: 0.9956
Testing Accuracy: 0.9855


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Pipeline ensures vectorizer is fitted ONLY on training data in each fold
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('svm', SVC(kernel='linear'))
])

# Run cross-validation on RAW TEXT (not pre-vectorized)
# This prevents vocabulary leakage across folds
X_raw = df['Message']
y = df['Result']

scores = cross_val_score(pipeline, X_raw, y, cv=5, scoring='accuracy')

print("=" * 70)
print("CROSS-VALIDATION RESULTS (No Data Leakage)")
print("=" * 70)
print(f"Fold Scores: {scores}")
print(f"Mean Accuracy: {scores.mean():.4f}")
print(f"Std Dev: {scores.std():.4f}")
print("Each fold trains vectorizer independently - no vocabulary leakage.")

CROSS-VALIDATION RESULTS (No Data Leakage)
Fold Scores: [0.98355899 0.98259188 0.98065764 0.9767667  0.98257502]
Mean Accuracy: 0.9812
Std Dev: 0.0024
Each fold trains vectorizer independently - no vocabulary leakage.


In [15]:
# 1. Define your new message(s) in a list
new_messages = ["Get ready to be a part of the 1 Lakh+ job seekers who've already taken the first step to achieving their dream jobs with our help. And guess what? We've generated 600,000+ resume scans, helping folks like you pinpoint exactly where they can shine brighter!"]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Get ready to be a part of the 1 Lakh+ job seekers who've already taken the first step to achieving their dream jobs with our help. And guess what? We've generated 600,000+ resume scans, helping folks like you pinpoint exactly where they can shine brighter! 
Result: NOT SPAM

Confidence Score: [0.75765656]


In [16]:
# 1. Define your new message(s) in a list
new_messages = ["Dear Student,Congratulations!You have been shortlisted for the Online Internship Program powered by Microsoft, IBM, and Meta, in collaboration with MINDENIOUS. This exclusive opportunity is available for batches commencing during the Semester Break in January, February, March, and April.Please note that limited scholarship seats are allocated for your college. We strongly recommend reserving your spot at the earliest to avoid missing this opportunity."]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result 
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Dear Student,Congratulations!You have been shortlisted for the Online Internship Program powered by Microsoft, IBM, and Meta, in collaboration with MINDENIOUS. This exclusive opportunity is available for batches commencing during the Semester Break in January, February, March, and April.Please note that limited scholarship seats are allocated for your college. We strongly recommend reserving your spot at the earliest to avoid missing this opportunity. 
Result: NOT SPAM

Confidence Score: [1.19268414]


In [17]:
# 1. Define your new message(s) in a list
new_messages = ["Hi Steel Fabrication Works,I noticed that searches for steel rod dealers on Justdial are up 43% this week. Since your business is a part of this category, I wanted to flag it for you.It may be a good time to review your listing to make the most of the trend."]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Hi Steel Fabrication Works,I noticed that searches for steel rod dealers on Justdial are up 43% this week. Since your business is a part of this category, I wanted to flag it for you.It may be a good time to review your listing to make the most of the trend. 
Result: NOT SPAM

Confidence Score: [0.93574945]


In [18]:
# 1. Define your new message(s) in a list
new_messages = ["Dear Abhishek Vishwakarma,This is an important email containing all the details for your upcoming Internship Common Aptitude Test (iCAT) – your first step towards an exciting and prestigious internship journey with us! Please review the details carefully."]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Dear Abhishek Vishwakarma,This is an important email containing all the details for your upcoming Internship Common Aptitude Test (iCAT) – your first step towards an exciting and prestigious internship journey with us! Please review the details carefully. 
Result: NOT SPAM

Confidence Score: [0.91804605]


In [19]:
# 1. Define your new message(s) in a list
new_messages = ["ear ABHISHEK KUMAR VISHWAKARMA ,Congratulations! You have been selected for the AICTE Oasis Infobyte Internship.Welcome Letter: Check Your Offer LetterUseful Links:Task Submission Form: Submission Form LinkTask List: Task List PDF LinkFAQs: FAQ PDF LinkPlacement Support: Placement Support PDF LinkResume Building Training: YouTube.comOasisInfobyteImportant Links: Click HereWe invite you to collaborate in open-source industry projects and gain real-world experience. For updates, join our community Telegram group here: https://t.me/oasisinfobyte Internship Batch Details:Project Kick-Off: Please begin your project as soon as possible.Start Date: 05th February 202Submission Deadline: 15th March 2026Internship Guidelines:Update your LinkedIn profile and share your achievements (Offer Letter, Completion Certificate, etc.). Tag Oasis Infobyte and use #oasisinfobyte.Plagiarized or copied work will result in internship termination and blacklisting from future opportunities.Share a demonstration video of your completed tasks on LinkedIn with a proper explanation. Tag Oasis Infobyte and use #oasisinfobyte.Technical interns should maintain a GitHub repository named OIBSIP for all tasks and submit the repo link in the submission form."]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: ear ABHISHEK KUMAR VISHWAKARMA ,Congratulations! You have been selected for the AICTE Oasis Infobyte Internship.Welcome Letter: Check Your Offer LetterUseful Links:Task Submission Form: Submission Form LinkTask List: Task List PDF LinkFAQs: FAQ PDF LinkPlacement Support: Placement Support PDF LinkResume Building Training: YouTube.comOasisInfobyteImportant Links: Click HereWe invite you to collaborate in open-source industry projects and gain real-world experience. For updates, join our community Telegram group here: https://t.me/oasisinfobyte Internship Batch Details:Project Kick-Off: Please begin your project as soon as possible.Start Date: 05th February 202Submission Deadline: 15th March 2026Internship Guidelines:Update your LinkedIn profile and share your achievements (Offer Letter, Completion Certificate, etc.). Tag Oasis Infobyte and use #oasisinfobyte.Plagiarized or copied work will result in internship termination and blacklisting from future opportunities.Share a demon

In [20]:
# 1. Define your new message(s) in a list
new_messages = ["Dear Customer, Thank you for banking with us. We hope that your experience of banking with us has been seamless and as a part of our continuous efforts to provide simplified banking, we are pleased to send you the Statement of your Loan Account.In order to ensure maximum privacy and confidentiality, the attached Statement of Account is password protected."]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Dear Customer, Thank you for banking with us. We hope that your experience of banking with us has been seamless and as a part of our continuous efforts to provide simplified banking, we are pleased to send you the Statement of your Loan Account.In order to ensure maximum privacy and confidentiality, the attached Statement of Account is password protected. 
Result: NOT SPAM

Confidence Score: [0.03096301]


In [21]:
# 1. Define your new message(s) in a list
new_messages = ["Dear Customer,Greetings from Punjab National Bank..!Kindly find attached the monthly statement of your Accounts from 01-Jan-2026 to 31-Jan-2026 in pdf format. To open this file, you need Adobe Acrobat Reader. You can download free version from the link http://www.adobe.com/products/acrobat/readstep2.html"]

# 2. Transform the text into numbers (Use .transform, NOT .fit_transform)
new_messages_numeric = vectorizer.transform(new_messages)

# 3. Make the prediction
predictions = model.predict(new_messages_numeric)

# 4. Show the result
for msg, label in zip(new_messages, predictions):
    status = "NOT SPAM" if label == 1 else "SPAM"
    print(f"Message: {msg} \nResult: {status}\n")
# Shows how far the message is from the 'boundary'
# Positive numbers usually mean Class 1 (NOT SPAM), negative mean Class 0 (SPAM)
confidence = model.decision_function(new_messages_numeric)
print(f"Confidence Score: {confidence}")

Message: Dear Customer,Greetings from Punjab National Bank..!Kindly find attached the monthly statement of your Accounts from 01-Jan-2026 to 31-Jan-2026 in pdf format. To open this file, you need Adobe Acrobat Reader. You can download free version from the link http://www.adobe.com/products/acrobat/readstep2.html 
Result: SPAM

Confidence Score: [-0.665199]
